In [8]:
import pandas as pd
import numpy as np
import itertools as it
rng = np.random.default_rng(42)

Data:

In [2]:
n_items = 20
df = pd.read_csv(f'/Users/tailai/multiobjective/data/test_data_n20_exp5.csv')

items_df_visual = pd.DataFrame()
for trial in range(len(df)):
    trial_data = []
    for item in range(1, n_items+1): 
        row = {
            'Science': df[f'value1_item_{item}'][trial],
            'Culture': df[f'value2_item_{item}'][trial],
            'Govern': df[f'value3_item_{item}'][trial],
            'Money': df[f'weight1_item_{item}'][trial],
            'Space': df[f'weight2_item_{item}'][trial],
            'capacity1': df['capacity1'][trial],
            'capacity2': df['capacity2'][trial],
        }
        trial_data.append(row)
    items_df_visual = pd.concat([items_df_visual, pd.DataFrame(trial_data)], ignore_index=True) ## create a new sequential index

test_trial = items_df_visual.iloc[:20,:]

In [3]:
test_trial

,Science,Culture,Govern,Money,Space,capacity1,capacity2
0,0,2,4,-5,-4,-36,-17
1,5,1,2,-4,-2,-36,-17
2,4,3,2,-3,-2,-36,-17
3,2,3,1,-1,-2,-36,-17
4,3,1,1,-2,-2,-36,-17
5,3,1,1,-2,-2,-36,-17
6,0,2,4,-2,-2,-36,-17
7,0,0,1,-2,-4,-36,-17
8,2,2,2,-4,-3,-36,-17
9,1,1,2,-3,-3,-36,-17


Exact Pareto optimal solutions:

In [42]:
n_selected = 5
objectives = []
solutions = []

for combo in it.combinations(range(n_items), n_selected):
    solution = test_trial.iloc[list(combo)]

    # test if the solution exceeds capacity
    if np.abs(solution["Money"].sum()) > np.abs(solution["capacity1"].iloc[0]) or \
          np.abs(solution["Space"].sum()) > np.abs(solution["capacity2"].iloc[0]):
        continue

    # computes objective values
    objective = [
        solution["Science"].sum(),
        solution["Culture"].sum(),
        solution["Govern"].sum()
    ]
    objectives.append(objective)

    solutions.append(combo)

objectives = np.array(objectives) # shape = (num of solutions, num of objectives)
solutions = np.array(solutions) # shape = (num of solutions, num of items)

In [ ]:
# def non_dominated(objectives):
#     pop_size = np.shape(objectives)[0]
#     non_dominated = np.zeros(pop_size, dtype=bool)
#     for i in range(pop_size):
#         dominated = np.zeros(pop_size, dtype=bool)
#         for j in range(pop_size):
#             # dominated[j]= np.all(objectives[i,:] < objectives[j,:])
#             dominated[j]= np.all(objectives[j,:] >= objectives[i,:]) and \
#                           np.any(objectives[j,:] > objectives[i,:])
#         non_dominated[i] = ~np.any(dominated)
#     return non_dominated

# pareto = non_dominated(objectives)

In [21]:
def non_dominated(objectives):
    n_solutions = objectives.shape[0]
    non_dominated = np.ones(n_solutions, dtype=bool)
    for i in range(n_solutions):
        for j in range(n_solutions):
            if i == j:
                continue

            if np.all(objectives[j, :] >= objectives[i, :]) and \
                np.any(objectives[j, :] > objectives[i, :]):
                non_dominated[i] = False
                break

    return non_dominated

pareto = non_dominated(objectives)

EDA:

In [30]:
def evaluatePopulation(population, n_obj):
    objectives = np.zeros((np.shape(population)[0], n_obj))
    for j in range(np.shape(population)[0]):
        solution = population[j, :]
        objectives[j, :] = np.sum(test_trial.iloc[solution, :3].values, axis=0)
    return objectives

def samplePopulation(distribution, money_cap, space_cap, pop_size, n_selected):
    pop_count = 0
    population = np.zeros((pop_size, n_selected), dtype=int)
    n_cards = np.size(distribution)

    while pop_count < pop_size:
        sample = rng.choice(n_cards, n_selected, p=distribution, replace=False).astype(int)
        if np.abs(np.sum(test_trial.iloc[sample, 3].values)) <= np.abs(money_cap) or \
            np.abs(np.sum(test_trial.iloc[sample, 4].values)) <= np.abs(space_cap):
            population[pop_count, :] = sample
            pop_count += 1
            
    return population

In [ ]:
n_cards = n_items
n_selected = 5
n_obj = 3
money_cap = test_trial["capacity1"].iloc[0]
space_cap = test_trial["capacity2"].iloc[0]
pop_size = 1000
training_size = 10


distribution = np.ones(n_cards)/n_cards
population = samplePopulation(distribution, money_cap, space_cap, pop_size, n_selected) 
objectives = evaluatePopulation(np.array(population), n_obj)
pareto = non_dominated(objectives)
pareto_front = objectives[pareto,:]
pareto_solutions = population[pareto,:]

distribution = np.ones(n_cards)
distribution += np.bincount(pareto_solutions.flatten(), minlength=n_cards)
distribution /= np.sum(distribution)

for j in range(training_size):
    prev_distribution = distribution.copy()
    
    population = samplePopulation(distribution, money_cap, space_cap, pop_size, n_selected)
    objectives = evaluatePopulation(np.array(population), n_obj)
    objectives = np.vstack((pareto_front, objectives))
    population = np.vstack((pareto_solutions, population))
    pareto = non_dominated(objectives)
    pareto_front = objectives[pareto,:]
    pareto_solutions = population[pareto,:]

    distribution = np.ones(n_cards)
    distribution += np.bincount(pareto_solutions.flatten(), minlength=n_cards)
    distribution /= np.sum(distribution)

    print(f"Iteration {j+1}")

    if np.allclose(distribution, prev_distribution, atol=1e-6):
        print("Converged")
        break

Iteration 1:
Iteration 2:
Iteration 3:
Iteration 4:
Iteration 5:
Iteration 6:
Iteration 7:
Iteration 8:
Iteration 9:
Iteration 10:


In [33]:
print("Pareto front size:", np.shape(pareto_front)[0])

Pareto front size: 4819
